<a href="https://colab.research.google.com/github/mechhector/visao_computacional20212/blob/main/trabalho2/Q1__frequency_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Trabalho 2 - DIP 2021.2
# Hector Leonardo Mota Moreira

# Filtering in the frequency domain

import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
from google.colab import files


# load the image

img = cv2.imread('/content/drive/MyDrive/Computer_Vision/Trabalho_1 /img_scr/messi.jpg',0)

cv2_imshow(img)

In [ ]:
# Fourier transform and quadrant shift with numpy

f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = 20*np.log(np.abs(fshift))

cv2_imshow(magnitude_spectrum)   



In [ ]:
# HIGH PASS FILTER

rows, cols = img.shape
crow,ccol = round(rows/2) , round(cols/2)
fshift[crow-30:crow+30, ccol-30:ccol+30] = 1
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

cv2_imshow(img_back)

In [ ]:
# Low pass filter ; exemple from the openCV tutorials

dft = cv2.dft(np.float32(img),flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))

rows, cols = img.shape
crow,ccol = round(rows/2) , round(cols/2)

# create a mask first, center square is 1, remaining all zeros
mask = np.zeros((rows,cols,2),np.uint8)
mask[crow-30:crow+30, ccol-30:ccol+30] = 1

# apply mask and inverse DFT
fshift = dft_shift*mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:,:,0],img_back[:,:,1])


plt.imshow(img_back, cmap = 'gray')
plt.figure(figsize = (150,150))

plt.show()

In [ ]:
# notch filter

img = cv2.imread('/content/lena.jpeg',0)

# get min and max values of img
img_min, img_max = np.amin(img, (0,1)), np.amax(img, (0,1))

print(img_min,img_max)

# convert image to floats and do dft saving as complex output
dft = cv2.dft(np.float32(img), flags = cv2.DFT_COMPLEX_OUTPUT)

# apply shift of origin from upper left corner to center of image
dft_shift = np.fft.fftshift(dft)

# extract magnitude and phase images
mag, phase = cv2.cartToPolar(dft_shift[:,:,0], dft_shift[:,:,1])

# get spectrum
spec = np.log(mag) / 20

# create mask from spectrum keeping only the brightest spots as the notches
mask = cv2.normalize(spec, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
mask = cv2.threshold(mask, 0.65, 1, cv2.THRESH_BINARY)[1]

# dilate mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

# cover center DC component by circle of black leaving only a few white spots on black background
xcenter = mask.shape[1] // 2
ycenter = mask.shape[0] // 2
mask = cv2.circle(mask, (xcenter,ycenter), radius=20, color=0, 
                                            thickness=cv2.FILLED)

# apply mask to magnitude such that magnitude is made zero where mask is one, 
#ie at spots
mag[mask!=0] = 0

# convert new magnitude and old phase into cartesian real and imaginary 
#components
real, imag = cv2.polarToCart(mag, phase)

# combine cartesian components into one complex image
back = cv2.merge([real, imag])

# shift origin from center to upper left corner
back_ishift = np.fft.ifftshift(back)

# do idft saving as complex output
img_back = cv2.idft(back_ishift)

# combine complex components into original image again
img_back = cv2.magnitude(img_back[:,:,0], img_back[:,:,1])

# re-normalize to 8-bits in range of original
min, max = np.amin(img_back, (0,1)), np.amax(img_back, (0,1))
print(min,max)
notched = cv2.normalize(img_back, None, alpha=img_min, beta=img_max, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

cv2_imshow(img)
